# How to use multiple sourcemaps in pyELQ
This example is intended to provide an overview on how to use the functionality of estimating multiple sourcemaps simultaneously. If this is your first time using pyELQ we advise to take a look at the basic example first. We will skip all parts which are covered by that example. 

We first generate some data and next set up 3 sourcemaps:
- One sourcemap with fixed sources for which we only estimate the emission rate, the source location is fixed
- Two sourcemaps using the reversible jump (RJ) algorithm which estimates bot emission rate as source location simultaneously. The 2 RJ sourcemaps have different site limits to show the possibility of this implementation.

Finally we plot the various results.

## CAUTION
Please keep in mind that pyELQ analyses all provided sourcemaps simultaneously, therefore when providing overlapping sourcemaps there might be ambiguities between the overlapping areas as multiple solutions (effectively a linear combination of the different overlapping maps) can explain the same information observed in the data. It is therefore key to really investigate the results and use overlapping sourcemaps with caution.

In [ ]:
import datetime
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pyelq.component.background import SpatioTemporalBackground
from pyelq.component.error_model import BySensor
from pyelq.component.offset import PerSensor
from pyelq.component.source_model import Normal
from pyelq.coordinate_system import ENU, LLA
from pyelq.dispersion_model.gaussian_plume import GaussianPlume
from pyelq.gas_species import CH4
from pyelq.model import ELQModel
from pyelq.meteorology import Meteorology
from pyelq.plotting.plot import Plot
from pyelq.preprocessing import Preprocessor
from pyelq.sensor.beam import Beam
from pyelq.sensor.sensor import Sensor, SensorGroup
from pyelq.source_map import SourceMap
from pyelq.support_functions.post_processing import calculate_rectangular_statistics

time_axis = pd.array(
    pd.date_range(start="2024-01-01 08:00:00", end="2024-01-01 12:00:00", freq="120s"), dtype="datetime64[ns]"
)
nof_observations = time_axis.size
reference_latitude = 0
reference_longitude = 0
reference_altitude = 0

radius = 30
angles = np.linspace(0, 90, 5)
sensor_x = radius * np.cos(angles * np.pi / 180)
sensor_y = radius * np.sin(angles * np.pi / 180)
sensor_z = np.ones_like(sensor_x) * 5.0

ENU_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)
ENU_object.from_array(np.vstack([sensor_x, sensor_y, sensor_z]).T)
LLA_object = ENU_object.to_lla()
LLA_array = LLA_object.to_array()

nof_sensors = LLA_array.shape[0]
sensor_group = SensorGroup()
for sensor in range(nof_sensors):
    new_sensor = Beam()
    new_sensor.label = f"Beam sensor {sensor}"
    new_sensor.location = LLA(
        latitude=np.array([reference_latitude, LLA_object.latitude[sensor]]),
        longitude=np.array([reference_longitude, LLA_object.longitude[sensor]]),
        altitude=np.array([5.0, LLA_object.altitude[sensor]]),
    )

    new_sensor.time = time_axis
    new_sensor.concentration = np.zeros(nof_observations)
    sensor_group.add_sensor(new_sensor)

sensor_x = np.array([5, 20])
sensor_y = np.array([22, 5])
sensor_z = np.ones_like(sensor_x) * 1.0
ENU_object = ENU(ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude)
ENU_object.from_array(np.vstack([sensor_x, sensor_y, sensor_z]).T)
LLA_object = ENU_object.to_lla()
LLA_array = LLA_object.to_array()

nof_sensors = LLA_array.shape[0]
for sensor in range(nof_sensors):
    new_sensor = Sensor()
    new_sensor.label = f"Point sensor {sensor}"
    new_sensor.location = LLA(
        latitude=np.array([LLA_object.latitude[sensor]]),
        longitude=np.array([LLA_object.longitude[sensor]]),
        altitude=np.array([LLA_object.altitude[sensor]]),
    )

    new_sensor.time = time_axis
    new_sensor.concentration = np.zeros(nof_observations)
    sensor_group.add_sensor(new_sensor)

met_object = Meteorology()

met_object.time = time_axis
met_object.wind_direction = np.linspace(0.0, 90.0, nof_observations) + np.random.normal(
    loc=0.0, scale=0.1, size=nof_observations
)
met_object.wind_speed = 4.0 * np.ones_like(met_object.wind_direction) + np.random.normal(
    loc=0.0, scale=0.1, size=nof_observations
)

met_object.calculate_uv_from_wind_speed_direction()

met_object.temperature = (273.1 + 15.0) * np.ones_like(met_object.wind_direction)
met_object.pressure = 101.325 * np.ones_like(met_object.wind_direction)

met_object.wind_turbulence_horizontal = 5.0 * np.ones_like(met_object.wind_direction)
met_object.wind_turbulence_vertical = 5.0 * np.ones_like(met_object.wind_direction)

source_map = SourceMap()
site_limits = np.array([[0, 30], [0, 30], [0, 10]])

location_object = ENU(
    ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
)

source_map.generate_sources(
    coordinate_object=location_object, sourcemap_limits=site_limits, sourcemap_type="hypercube", nof_sources=4
)

source_map.location.up = np.array([2.0, 3.0, 5.0, 1.5])
source_map.location.east = np.array([12.0, 20.0, 25.0, 7.0])
source_map.location.north = np.array([20.0, 15.0, 5.0, 25.0])


gas_object = CH4()
dispersion_model = GaussianPlume(source_map=deepcopy(source_map))
true_emission_rates = np.array([[15], [10], [5], [7]])
for current_sensor in sensor_group.values():
    coupling_matrix = dispersion_model.compute_coupling(
        sensor_object=current_sensor,
        meteorology_object=met_object,
        gas_object=gas_object,
        output_stacked=False,
        run_interpolation=False,
    )
    source_contribution = coupling_matrix @ true_emission_rates
    observation = (
        source_contribution.flatten()
        + 2.0
        + np.random.normal(loc=0.0, scale=0.01, size=current_sensor.nof_observations)
    )
    current_sensor.concentration = observation

analysis_time_range = [datetime.datetime(2024, 1, 1, 8, 0, 0), datetime.datetime(2024, 1, 1, 12, 0, 0)]

smoothing_period = 10 * 60

time_bin_edges = pd.array(
    pd.date_range(analysis_time_range[0], analysis_time_range[1], freq=f"{smoothing_period}s"), dtype="datetime64[ns]"
)

prepocessor_object = Preprocessor(
    time_bin_edges=time_bin_edges, sensor_object=sensor_group, met_object=met_object, aggregate_function="median"
)

min_wind_speed = 0.05
prepocessor_object.filter_on_met(filter_variable=["wind_speed"], lower_limit=[min_wind_speed], upper_limit=[np.inf])

In [ ]:
fig = go.Figure()
fig = sensor_group.plot_sensor_location(fig=fig)
fig.update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
fig.add_trace(
    go.Scattermap(
        mode="markers",
        lon=source_map.location.to_lla().longitude,
        lat=source_map.location.to_lla().latitude,
        name="True Source Locations",
        marker=go.scattermap.Marker(color="green", size=10),
    )
)
fig.show()

fig = met_object.plot_polar_hist()
fig.update_layout(height=400, margin={"r": 0, "l": 0})
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_axis, y=met_object.wind_direction, mode="markers", name="Wind direction"))
fig.update_layout(height=400, margin={"r": 0, "l": 0}, title="Wind Direction [deg]")
fig.show()

fig = go.Figure()
fig = sensor_group.plot_timeseries(fig=fig)
fig.update_layout(height=800, margin={"r": 0, "t": 10, "l": 0, "b": 0})
fig.show()

fig = go.Figure()
fig = met_object.plot_polar_scatter(fig=fig, sensor_object=sensor_group)
fig.update_layout(height=400, margin={"r": 0, "l": 0})
fig.show()

fig = go.Figure()
fig = prepocessor_object.sensor_object.plot_timeseries(fig=fig)
fig.update_layout(height=800, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

# Setting up the sourcemaps
We set up a fixed sourcemap which contains 2 "known" emitter locations in (East, North, Up) coordinates: (20, 15, 3) and (25, 5, 5)

We set up two source map for the reversible jump algorithm. The source map referred to as source_map_rj_1 has the default site_limits defined previously. 
The second source map referred to as source_map_rj_2 has a different site_limits referred to as site_limits_2 and gives the possibility of focusing more on a specific area defined by the site_limits_2. 
We generate two inital source location estimates within the site limits for each RJ sourcemap respectively using the generate sources method.

In [ ]:
source_map_fixed_sources = SourceMap()
location_object = ENU(
    ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
)

source_map_fixed_sources.location = location_object
source_map_fixed_sources.location.up = np.array([3.0, 5.0])
source_map_fixed_sources.location.east = np.array([20.0, 25.0])
source_map_fixed_sources.location.north = np.array([15.0, 5.0])
dispersion_model_fixed_sources_1 = GaussianPlume(source_map_fixed_sources)

In [ ]:
source_map_rj_1 = SourceMap()
location_object = ENU(
    ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
)

source_map_rj_1.generate_sources(
    coordinate_object=location_object, sourcemap_limits=site_limits, sourcemap_type="hypercube", nof_sources=2
)

dispersion_model_rj_1 = GaussianPlume(source_map_rj_1)

In [ ]:
source_map_rj_2 = SourceMap()
site_limits_2 = np.array([[10, 15], [17, 22], [1, 3]])

location_object = ENU(
    ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
)

source_map_rj_2.generate_sources(
    coordinate_object=location_object, sourcemap_limits=site_limits_2, sourcemap_type="hypercube", nof_sources=2
)

dispersion_model_rj_2 = GaussianPlume(source_map_rj_2)

We set up the different parameters for our MCMC model. Since we defined 3 source maps, we need to defined 3 source models. We also set up a background model, offset model and error model to complete the MCMC setup.

In [ ]:
source_model_fixed_sources = Normal(label_string="fixed")
source_model_fixed_sources.individual_source_labels = ["source_a", "source_b"]
source_model_fixed_sources.emission_rate_mean = np.array([0], ndmin=1)
source_model_fixed_sources.initial_precision = np.array([1 / (2.5**2)], ndmin=1)
source_model_fixed_sources.reversible_jump = False  # SETTING THE RJ FLAG TO FALSE HERE!
source_model_fixed_sources.rate_num_sources = 1.0
source_model_fixed_sources.dispersion_model = dispersion_model_fixed_sources_1
source_model_fixed_sources.update_precision = False
source_model_fixed_sources.site_limits = site_limits
source_model_fixed_sources.coverage_detection = 0.1  # ppm
source_model_fixed_sources.coverage_test_source = 3.0  # kg/hr

source_model_rj_1 = Normal(label_string="rj_1")
source_model_rj_1.emission_rate_mean = np.array([0], ndmin=1)
source_model_rj_1.initial_precision = np.array([1 / (2.5**2)], ndmin=1)
source_model_rj_1.reversible_jump = True
source_model_rj_1.rate_num_sources = 1.0
source_model_rj_1.dispersion_model = dispersion_model_rj_1
source_model_rj_1.update_precision = False
source_model_rj_1.site_limits = site_limits
source_model_rj_1.coverage_detection = 0.1  # ppm
source_model_rj_1.coverage_test_source = 3.0  # kg/hr

source_model_rj_2 = Normal(label_string="rj_2")
source_model_rj_2.emission_rate_mean = np.array([0], ndmin=1)
source_model_rj_2.initial_precision = np.array([1 / (2.5**2)], ndmin=1)
source_model_rj_2.reversible_jump = True
source_model_rj_2.rate_num_sources = 1.0
source_model_rj_2.dispersion_model = dispersion_model_rj_2
source_model_rj_2.update_precision = False
source_model_rj_2.site_limits = site_limits_2
source_model_rj_2.coverage_detection = 0.1  # ppm
source_model_rj_2.coverage_test_source = 3.0  # kg/hr

source_model = [source_model_fixed_sources, source_model_rj_1, source_model_rj_2]

background = SpatioTemporalBackground()
background.n_time = None
background.mean_bg = 2.0
background.spatial_dependence = True
background.initial_precision = 1 / np.power(3e-4, 2)
background.precision_time_0 = 1 / np.power(0.1, 2)
background.spatial_correlation_param = 25.0
background.update_precision = False

offset_model = PerSensor()
offset_model.update_precision = False
offset_model.initial_precision = 1 / (0.001) ** 2

error_model = BySensor()
error_model.initial_precision = 1 / (0.1) ** 2
error_model.prior_precision_shape = 1e-2
error_model.prior_precision_rate = 1e-2

We create an instance of the ELQModel class which forms the interface with the MCMC repo and run the MCMC algorithm.

In [ ]:
elq_model = ELQModel(
    sensor_object=prepocessor_object.sensor_object,
    meteorology=prepocessor_object.met_object,
    gas_species=gas_object,
    background=background,
    error_model=error_model,
    offset_model=offset_model,
    source_model=source_model,
)

elq_model.n_iter = 10000

elq_model.initialise()

elq_model.to_mcmc()
elq_model.run_mcmc()
elq_model.from_mcmc()

# Plotting the different results

Now we want to plot the quantification results on map. For this we create an instance of the Plot object. We can use the `plot_quantification_results_on_map` method to plot the combined result of all sourcemaps present in the ELQmodel which is the default behaviour. If we want to plot a specific sourcemap we need to specify the `source_model_to_plot_key` argument.
This argument is a string with the name of the sourcemap which a user defines themselves through the `label_string` prepended with 'source_'. In this example we would therefore want to plot any of the following keys: `source_fixed`, `source_rj_1`, `source_rj_2`, `sources_combined` corresponding to their relative sourcemap or to the combined result. Note that these keys are also visible when looking at the component keys of the ELQ model instance.

Let's first plot the combined results. We only plot a selection of the available plots, more (diagnostic) plots are covered in the basic example.

In [ ]:
elq_model.components.keys()

In [ ]:
burn_in = 5000

plotter = Plot()

plotter.plot_quantification_results_on_map(
    model_object=elq_model,
    source_model_to_plot_key=None,
    bin_size_x=1,
    bin_size_y=1,
    normalized_count_limit=0.1,
    burn_in=burn_in,
)

plotter.plot_fitted_values_per_sensor(
    mcmc_object=elq_model.mcmc, sensor_object=elq_model.sensor_object, burn_in=burn_in
)

true_source_location_trace = go.Scattermap(
    mode="markers",
    lon=source_map.location.to_lla().longitude,
    lat=source_map.location.to_lla().latitude,
    name="True Source Locations",
    marker=go.scattermap.Marker(color="green", size=10),
)

site_limits_enu = ENU(
    ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
)
site_limits_for_plot = np.array(
    [
        [site_limits[0, 0], site_limits[1, 0], 0],
        [site_limits[0, 1], site_limits[1, 0], 0],
        [site_limits[0, 1], site_limits[1, 1], 0],
        [site_limits[0, 0], site_limits[1, 1], 0],
        [site_limits[0, 0], site_limits[1, 0], 0],
    ]
)
site_limits_enu.from_array(site_limits_for_plot)
site_limits_lla = site_limits_enu.to_lla()
site_limits_trace = go.Scattermap(
    mode="lines",
    fill="toself",
    lon=site_limits_lla.longitude,
    lat=site_limits_lla.latitude,
    marker={"color": "white"},
    name="Site limits",
)


site_limits_2_enu = ENU(
    ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
)
site_limits_2_for_plot = np.array(
    [
        [site_limits_2[0, 0], site_limits_2[1, 0], 0],
        [site_limits_2[0, 1], site_limits_2[1, 0], 0],
        [site_limits_2[0, 1], site_limits_2[1, 1], 0],
        [site_limits_2[0, 0], site_limits_2[1, 1], 0],
        [site_limits_2[0, 0], site_limits_2[1, 0], 0],
    ]
)
site_limits_2_enu.from_array(site_limits_2_for_plot)
site_limits_2_lla = site_limits_2_enu.to_lla()

site_limits_2_trace = go.Scattermap(
    mode="lines",
    fill="toself",
    lon=site_limits_2_lla.longitude,
    lat=site_limits_2_lla.latitude,
    marker={"color": "grey"},
    name="Site limits 2",
)

In [ ]:
plotter.figure_dict["fitted_values"].update_layout(height=800, margin={"r": 0, "t": 50, "l": 0, "b": 0}).show()

In [ ]:
plotter.figure_dict["count_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter.figure_dict["count_map"].add_trace(true_source_location_trace)
plotter.figure_dict["count_map"].show()

In [ ]:
plotter.figure_dict["iqr_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter.figure_dict["iqr_map"].add_trace(true_source_location_trace)
plotter.figure_dict["iqr_map"].show()

In [ ]:
plotter.figure_dict["median_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter.figure_dict["median_map"].add_trace(true_source_location_trace)
plotter.figure_dict["median_map"].show()

Now, we plot quantification results on map for the fixed sourcemap, "source_fixed". 

In [ ]:
plotter_fixed = Plot()

plotter_fixed.plot_quantification_results_on_map(
    model_object=elq_model,
    source_model_to_plot_key="source_fixed",
    bin_size_x=1,
    bin_size_y=1,
    normalized_count_limit=0.1,
    burn_in=burn_in,
)

In [ ]:
plotter_fixed.figure_dict["median_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter_fixed.figure_dict["median_map"].add_trace(true_source_location_trace)
plotter_fixed.figure_dict["median_map"].show()

Lastly, we plot some quantification results on map for the two different RJ sourcemaps.

In [ ]:
plotter_rj_1 = Plot()

plotter_rj_1.plot_quantification_results_on_map(
    model_object=elq_model,
    source_model_to_plot_key="source_rj_1",
    bin_size_x=1,
    bin_size_y=1,
    normalized_count_limit=0.1,
    burn_in=burn_in,
    show_fixed_source_locations=False,
)

plotter_rj_1.figure_dict["median_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter_rj_1.figure_dict["median_map"].add_trace(true_source_location_trace)
plotter_rj_1.figure_dict["median_map"].add_trace(site_limits_trace)
plotter_rj_1.figure_dict["median_map"].add_trace(site_limits_2_trace)
plotter_rj_1.figure_dict["median_map"].show()

In [ ]:
plotter_rj_2 = Plot()

plotter_rj_2.plot_quantification_results_on_map(
    model_object=elq_model,
    source_model_to_plot_key="source_rj_2",
    bin_size_x=1,
    bin_size_y=1,
    normalized_count_limit=0.1,
    burn_in=burn_in,
    show_fixed_source_locations=False,
)

plotter_rj_2.figure_dict["median_map"].update_layout(
    map_style="open-street-map",
    map_center=dict(lat=reference_latitude, lon=reference_longitude),
    map_zoom=18,
    height=800,
    margin={"r": 0, "l": 0, "b": 0},
)
plotter_rj_2.figure_dict["median_map"].add_trace(true_source_location_trace)
plotter_rj_2.figure_dict["median_map"].add_trace(site_limits_trace)
plotter_rj_2.figure_dict["median_map"].add_trace(site_limits_2_trace)

plotter_rj_2.figure_dict["median_map"].show()

For completeness we calculate the rectangular statistics here for the combined sourcemap as well as the individual sourcemaps and print them as well as the true values. Note that the true sources and estimated sources are not in in the same order, hence this is just meant for qualitative comparison next to the comparison through the plots above. Always be cautious when using overlapping sourcemaps like the example we provided.

In [ ]:
for sourcemap_label in ["sources_combined", "source_fixed", "source_rj_1", "source_rj_2"]:
    source_locations = elq_model.components[sourcemap_label].all_source_locations
    emission_rates = elq_model.components[sourcemap_label].emission_rate

    _, _, _, _, _, summary_result = calculate_rectangular_statistics(
        emission_rates=emission_rates,
        source_locations=source_locations,
        bin_size_x=1,
        bin_size_y=1,
        normalized_count_limit=0.1,
        burn_in=burn_in,
    )
    if sourcemap_label == "sources_combined":
        combined_results_lla = LLA()
        combined_results_lla.from_array(array=summary_result.iloc[:, :3].values)
        combined_results_ENU = combined_results_lla.to_enu(
            ref_latitude=reference_latitude, ref_longitude=reference_longitude, ref_altitude=reference_altitude
        )
        print("True locations (E, N, U):")
        print(source_map.location.to_array())
        print("\nTrue emission rates:")
        print(true_emission_rates.flatten())
        print("\nCombined sources location estimates (E, N, U):")
        print(np.round(combined_results_ENU.to_array(), 2))
        print("\nCombined sources emission rate estimates:")
        print(np.round(summary_result.iloc[:, 4].values, 2))

    print(f"\nSummary results for {sourcemap_label}:\n")
    print(summary_result.iloc[:, :5])